In [36]:
!pip install langchain-text-splitters langchain pymupdf langchain_ollama psycopg2-binary pgvector numpy lxml selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 6.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 3.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 452.7 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.2/63.2 kB 495.4 kB/s eta 0:00:00 0:00:01
  Attempting uninstall: websocket-client
    Found existing installation: websocket-client 1.6.4
    Uninstalling websocket-client-1.6.4:
      Successfully uninstalled websocket-client-1.6.4
  Attempting uninstall: attrs
    Found existing installation: attrs 23.1.0
    Uninstalling attrs-23.1.0:
      Successfully uninstalled attrs-23.1.0


In [30]:
import urllib.request
import hashlib
from io import BytesIO

import pymupdf  # PyMuPDF
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, 
    chunk_overlap=50,  
    separators=["\n\n", "\n", " ", ""],  # Prioritize logical breaks
)

from langchain_ollama import OllamaEmbeddings
embed = OllamaEmbeddings(
    model="all-minilm",
    base_url = "http://host.docker.internal:11434"
)
pg_pool = pool.SimpleConnectionPool(
    minconn=cores,  # Minimum connections
    maxconn=cores, # Maximum connections
    dsn=f"dbname={db} user={user} password={password} host={host}"
)

# Load the PDF
url = "https://assets.openstax.org/oscms-prodcms/media/documents/Calculus_Volume_1_-_WEB_68M1Z5W.pdf"  # PDF path or URL

stream = BytesIO(urllib.request.urlopen(url).read())
doc = pymupdf.open(stream=stream, filetype="pdf")
print(len(doc))
# for page in doc:
#     chunks = text_splitter.split_text(page.get_text())
#     embeddings = embed.embed_documents(chunks)
#     print(len(chunks))
#     print(len(embeddings))
#     for chunk_id, (chunk, embedding) in enumerate(zip(chunks,embeddings)):
#         print(f"Page: {page.number}, Chunk: {chunk_id}, Content: {chunk}")#, Embedding: {embedding}")
#         print("---")

875


In [27]:
from pgvector.psycopg2 import register_vector
import numpy as np
from langchain_ollama import OllamaEmbeddings
embed = OllamaEmbeddings(
    model="all-minilm",
    base_url = "http://host.docker.internal:11434"
)

search = np.array(embed.embed_query('hello'))
from psycopg2 import pool
db = 'search'
user = 'postgres'
password = 'Washington17354!'
host = '192.168.0.102'
query = """SELECT a.id, a.page, b.uri, b.title, b.author, a.embedding <=> %s AS rank
FROM semantic_search a
INNER JOIN sources b ON a.id = b.id
ORDER BY rank
LIMIT %s"""
limit = 5
pg_pool = pool.SimpleConnectionPool(
    minconn=1,  # Minimum connections
    maxconn=4, # Maximum connections
    dsn=f"dbname={db} user={user} password={password} host={host}"
)
with pg_pool.getconn() as con:
    register_vector(con)
    with con.cursor() as cur:
        cur.execute(query,(search,limit,));
        result = cur.fetchone()
        print(result);
        
    

('d3411b6a43b2a6300108a85a3270b468', 10, 'https://arxiv.org/pdf/2502.15279', None, None, 0.72788135575541)


In [18]:
import requests
from bs4 import BeautifulSoup
import time


# First we want to get all the identifiers we're going to process
URL = "http://export.arxiv.org/oai2?set=cs&verb=ListIdentifiers&metadataPrefix=oai_dc&from=2025-03-08"
page = requests.get(URL)

oai_soup = BeautifulSoup(page.content, "xml")
ids = oai_soup.find_all("identifier")
texts = [id.get_text().replace('oai:arXiv.org:','https://arxiv.org/abs/') for id in ids]
print(texts[:10])



['https://arxiv.org/abs/1704.08000', 'https://arxiv.org/abs/2012.10711', 'https://arxiv.org/abs/2101.11003', 'https://arxiv.org/abs/2104.11977', 'https://arxiv.org/abs/2106.05421', 'https://arxiv.org/abs/2204.08027', 'https://arxiv.org/abs/2209.12526', 'https://arxiv.org/abs/2210.01242', 'https://arxiv.org/abs/2210.16010', 'https://arxiv.org/abs/2212.11805']
A Framework for Algorithm Stability


NameError: name 'trim' is not defined

In [32]:
for uri in texts[:1]:
    time.sleep(3)
    arxiv_page = requests.get(uri)
    soup = BeautifulSoup(arxiv_page.content, "html.parser")
    print(soup.find("h1",class_='title mathjax').get_text().replace('Title:',''))
    print(soup.find("blockquote", class_='abstract mathjax').get_text().replace('Abstract:','').strip())
    print('\n'.join([a.get_text() + " (" + a.get('href') + ")" for a in soup.find("div",class_='authors').find_all('a')]))
    print('https://arxiv.org' + soup.find('a',class_='download-pdf').get('href'))

A Framework for Algorithm Stability
We say that an algorithm is stable if small changes in the input result in small changes in the output. This kind of algorithm stability is particularly relevant when analyzing and visualizing time-varying data. Stability in general plays an important role in a wide variety of areas, such as numerical analysis, machine learning, and topology, but is poorly understood in the context of (combinatorial) algorithms. In this paper we present a framework for analyzing the stability of algorithms. We focus in particular on the trade-off between the stability of an algorithm and the quality of the solution it computes. Our framework allows for three types of stability analysis with increasing degrees of complexity: event stability, topological stability, and Lipschitz stability. In addition, we need to refine the model of an algorithm based on how it interacts with the time-varying data, for which we consider several options. We demonstrate the use of our st

In [37]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Chrome()  # Or other browser driver
driver.get('https://openstax.org/details/books/calculus-volume-1')

try:
    # Wait for specific elements to load (e.g., using XPath)
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//div[@class='loc-summary-text']"))
    )
    html_content = driver.page_source
    print(html_content)
finally:
    driver.quit()

soup = BeautifulSoup(html_content, "html.parser")
print(soup)
print(soup.find("h1",class_='image-heading'))#.find('img').get('alt'))
print(soup.find("div", class_='loc-summary-text').find('p').get_text())
print('\n'.join(soup.find_all('div', class_='loc-senior-author')) +'\n' + '\n'.join(soup.find_all('div', class_='loc-nonsenior-author')))
print(soup.find('a',string='Download a PDF').get('href'))

WebDriverException: Message: Service /home/jovyan/.cache/selenium/chromedriver/linux64/134.0.6998.35/chromedriver unexpectedly exited. Status code was: 127


In [27]:
from queue import Queue
import multiprocessing
from threading import Thread
from psycopg2 import pool
import time
import hashlib
from io import BytesIO
import io
import random
import requests
import pymupdf  # PyMuPDF
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from bs4 import BeautifulSoup


host = "192.168.0.102"
user = "postgres"
password = "Washington17354!"
db = "search"
oai_uri = "http://export.arxiv.org/oai2?set=cs&verb=ListIdentifiers&metadataPrefix=oai_dc&from=2025-03-08"
page = requests.get(oai_uri)
oai_soup = BeautifulSoup(page.content, "xml")
ids = oai_soup.find_all("identifier")
urls = [id.get_text().replace('oai:arXiv.org:','https://arxiv.org/abs/') for id in ids][:10]


sources_insert = "INSERT INTO sources (id, uri, title, author, summary) VALUES (%s, %s, %s, %s, %s)"
#"""INSERT INTO sources (id, uri, title, summary, title_embedding, summary_embedding)
#VALUES (%s, %s, %s, %s, %s, %s)"""
semantic_insert = "INSERT INTO semantic_search (id, page, chunk, embedding) VALUES (%s, %s, %s, %s)"
keyword_insert = "INSERT INTO keyword_search (id, page, ts) VALUES (%s, %s, to_tsvector('english', %s))"

def process_resource(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    if (url.contains('arxiv')):
        title = soup.find("h1",class_='title mathjax').get_text().replace('Title:','')
        summary = soup.find("blockquote", class_='abstract mathjax').get_text().replace('Abstract:','').strip()
        authors = '\n'.join([a.get_text() + " (" + a.get('href') + ")" for a in soup.find("div",class_='authors').find_all('a')])
        pdf_url = 'https://arxiv.org' + soup.find('a',class_='download-pdf').get('href')
    else:
        

def thread_task(inq,con,core,chunk_size=500, chunk_overlap=50):
    embed = OllamaEmbeddings(
        model="all-minilm",
        base_url = "http://host.docker.internal:11434"
    )
    # LangChain's Recursive Text Splitter
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, 
        chunk_overlap=chunk_overlap,  
        separators=["\n\n", "\n", " ", ""],  # Prioritize logical breaks
    )
    while True:
        with con.cursor() as cur:
            url = inq.get()
            if url is None:
                inq.task_done()
                con_queue.put(con)
                break
            print(f"Processing PDF: {url}")
            try:
                key = hashlib.md5(url.encode()).hexdigest()
                time.sleep(3)
                (title,summary,authors,pdf_url) = process_resource(url)
                with BytesIO() as stream_buffer:
                    response = requests.get(pdf_url, stream=True)
                    if response.status_code != 200:
                        raise Exception(f"Failed to download PDF; received HTTP {response.status_code} from underlying server")
                    for chunk in response.iter_content(chunk_size=io.DEFAULT_BUFFER_SIZE):
                        stream_buffer.write(chunk)
                    doc = pymupdf.open(stream=stream_buffer, filetype="pdf")
                    cur.execute(sources_insert,(key,pdf_url,title,authors,summary,))
                    for page in doc:
                        try:
                            cur.execute(keyword_insert,(key,page.number,page.get_text().replace('\x00', ''),))
                            chunks = text_splitter.split_text(page.get_text())
                            embeddings = embed.embed_documents(chunks)
                            for chunk_id, (chunk,embedding) in enumerate(zip(chunks,embeddings)):
                                cur.execute(semantic_insert,(key,page.number,chunk_id,embedding,))
                        except Exception as e:
                            print(f"Error processing page {page.number} of {url}: {e}")
                            continue  # Skip to the next page
            except Exception as e:
                print(f"Error downloading PDF from url: {url}: {e}")
                continue  # Skip to the next page

            con.commit()
            inq.task_done()
            time.sleep(3)

cores = multiprocessing.cpu_count()
insert_queue = Queue()

for url in urls:
    insert_queue.put(url)
# Create a connection pool
pg_pool = pool.SimpleConnectionPool(
    minconn=cores,  # Minimum connections
    maxconn=cores, # Maximum connections
    dsn=f"dbname={db} user={user} password={password} host={host}"
)
con_queue = Queue()
for core in range(0,cores):
    insert_queue.put(None)
    con_queue.put(None)

print("Starting insert of documents into Postgres")
insert_start_time = time.time()
join_list = []
for core in range(0,cores):
    worker = Thread(target=thread_task, args=(insert_queue,pg_pool.getconn(),core))
    worker.start()
    join_list.append(worker)
    time.sleep(3)
        
for thread in join_list:
    thread.join()

while not con_queue.empty():
    connection = con_queue.get()
    if connection is None:
        con_queue.task_done()
        break
    connection.close()
    con_queue.task_done()

insert_end_time = time.time()
print(f"Completed inserting documents in {insert_end_time - insert_start_time} seconds")

Starting insert of documents into Postgres
Processing PDF: https://arxiv.org/abs/1704.08000
Processing PDF: https://arxiv.org/abs/2012.10711
Processing PDF: https://arxiv.org/abs/2101.11003
Processing PDF: https://arxiv.org/abs/2104.11977
Processing PDF: https://arxiv.org/abs/2106.05421
Processing PDF: https://arxiv.org/abs/2204.08027
Processing PDF: https://arxiv.org/abs/2209.12526
Processing PDF: https://arxiv.org/abs/2210.01242
Processing PDF: https://arxiv.org/abs/2210.16010
Processing PDF: https://arxiv.org/abs/2212.11805
Completed inserting documents in 40.0526180267334 seconds


In [ ]:
import psycopg2

conn = psycopg2.connect("dbname=mydb user=myuser password=mypass host=localhost")
cur = conn.cursor()

cur.execute("""
    INSERT INTO sources (id, uri, title, summary, title_embedding, summary_embedding)
    VALUES (%s, %s, %s, %s, %s, %s)
""", (doc_id, "path/to/file.pdf", title, summary, title_embedding, summary_embedding))

conn.commit()

In [5]:
# for page_num, page in enumerate(doc.pages):
#     chunks = page.chunk()  # Split into smaller parts

#     for chunk_idx, chunk in enumerate(chunks):
#         embedding = doc.embed(chunk.text)

#         # Insert into semantic_search
#         cur.execute("""
#             INSERT INTO semantic_search (id, page, chunk, embedding)
#             VALUES (%s, %s, %s, %s)
#         """, (doc_id, page_num + 1, chunk_idx, embedding))

#         # Insert into keyword_search
#         cur.execute("""
#             INSERT INTO keyword_search (id, page, ts)
#             VALUES (%s, %s, to_tsvector('english', %s))
#         """, (doc_id, page_num + 1, chunk.text))

# conn.commit()
# cur.close()
# conn.close()

Token indices sequence length is longer than the specified maximum sequence length for this model (525 > 512). Running this sequence through the model will result in indexing errors


text="FRANK ELAVSKY, Carnegie Mellon University, fje@cmu.edu\nAcademic, peer-reviewed 'short' papers are a common way to present a late-breaking work to the academic community that outlines preliminary findings, research ideas, and novel conversations. By comparison, blogging or writing posts on social media are an unstructured and open way to discuss ideas and start new conversations. Both have limitations in the proliferation of research ideas. The short paper format relies on the conference and journal submission process while blogging does not operate within a structured format or set of expectations at all. However, at times the demand exists for late-breaking ideas and conversations to arise in a raw form or with urgency but should still be archived and recorded in a way that promotes citational honesty and integrity. To address this, I present: The Micro-Paper, as a micro-paper itself. The Micro-Paper is a small, cheap, accessible, digital document that is self-published and arc